In [ ]:
from urllib.request import urlopen
import pickle
import re

from bs4 import BeautifulSoup

# %pip install fsspec --user

In [ ]:
# download data

url = "https://es.wikipedia.org/wiki/Portal:Argentina/Efem%C3%A9rides"

html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

In [ ]:
from datetime import datetime

def converter_date(text, year):
    year = year

    months = {
        "enero": 1,
        "febrero": 2,
        "marzo": 3,
        "abril": 4,
        "mayo": 5,
        "junio": 6,
        "julio": 7,
        "agosto": 8,
        "septiembre": 9,
        "octubre": 10,
        "noviembre": 11,
        "diciembre": 12
    }

    words = text.split()
    day = int(words[0])
    
    try:
        month = words[1].lower() 

        date_str = f"{year}-{months[month]:02d}-{day:02d}"

        date = datetime.strptime(date_str, "%Y-%m-%d")
    except:
        pass


    return date

In [ ]:
# create Dict wiht events
data = {}

# regex para identificar filas con información
p = re.compile("[1][7-9][0-9][0-9] – ")

def clean_info(value):
    a = value[0:7]
    if p.match(a) is not None:
        return value

index = 0
for row, line in enumerate(text.split('\n')):
    a = line[0:7]
    if line[0:15] == 'Efemérides del ':
        b = line[15:].replace(" de ", " ")
    if p.match(a) is not None:
        data[index] = [converter_date(b, line[0:4]), line[7:]]
        index += 1

In [ ]:
# Save to file
with open('saved_dictionary.pkl', 'wb') as f:
    pickle.dump(data, f)
        
#with open('saved_dictionary.pkl', 'rb') as f:
#    data = pickle.load(f)